In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import sys
sys.path.insert(0,'../')

In [2]:
from testw2v import config, gensim_utils
global_conf = config.load('../config.yaml')

In [3]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten

In [40]:
file = '../wiki_10pct'

In [5]:
import logging
logging.basicConfig(level=logging.DEBUG)

## The Google Default

In [41]:
conf = global_conf['experiments']['google_default']

In [43]:
conf['batch_size'] = 64

In [44]:
conf['num_ns'] = 10

In [46]:
conf['sequence_length'] = 100
conf['window_size'] = 4

In [47]:
from testw2v import google_example

In [48]:
dataset, vector_layer = google_example.file_to_dataset(file, conf)

100%|██████████| 180137/180137 [00:40<00:00, 4500.23it/s]


In [49]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=conf['num_ns']+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)

In [50]:
def build_model():
    embedding_dim = conf['embedding_dim'] 
    word2vec = Word2Vec(conf['vocab_size'], embedding_dim)
    word2vec.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return word2vec


In [51]:
model = build_model()

In [52]:
model.fit(dataset, epochs=conf['epochs'])

Epoch 1/20
326/326 [==============================] - 3s 8ms/step - loss: 1.5604 - accuracy: 0.3491
Epoch 2/20
326/326 [==============================] - 2s 8ms/step - loss: 1.3427 - accuracy: 0.4601
Epoch 3/20
326/326 [==============================] - 2s 8ms/step - loss: 1.2516 - accuracy: 0.5120
Epoch 4/20
326/326 [==============================] - 2s 7ms/step - loss: 1.1773 - accuracy: 0.5505
Epoch 5/20
326/326 [==============================] - 2s 7ms/step - loss: 1.1109 - accuracy: 0.5828
Epoch 6/20
326/326 [==============================] - 3s 8ms/step - loss: 1.0498 - accuracy: 0.6113
Epoch 7/20
326/326 [==============================] - 3s 8ms/step - loss: 0.9927 - accuracy: 0.6371
Epoch 8/20
326/326 [==============================] - 3s 8ms/step - loss: 0.9389 - accuracy: 0.6607
Epoch 9/20
326/326 [==============================] - 2s 8ms/step - loss: 0.8882 - accuracy: 0.6828
Epoch 10/20
326/326 [==============================] - 2s 8ms/step - loss: 0.8405 - accuracy: 0.7042

In [53]:
vocab = vector_layer.get_vocabulary()
weights = model.get_layer('w2v_embedding').get_weights()[0]

gensim_obj = gensim_utils.w2v_to_gensim(vocab, weights)
google_metrics = gensim_utils.metrics(gensim_obj)

INFO:gensim.models.keyedvectors:precomputing L2-norms of word weight vectors
DEBUG:smart_open.smart_open_lib:{'transport_params': None, 'ignore_ext': False, 'opener': None, 'closefd': True, 'newline': None, 'errors': None, 'encoding': None, 'buffering': -1, 'mode': 'rb', 'uri': '/opt/conda/envs/emb3/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv'}
DEBUG:gensim.models.keyedvectors:Skipping line #3 with OOV words: tiger	cat	7.35
DEBUG:gensim.models.keyedvectors:Skipping line #4 with OOV words: tiger	tiger	10.00
DEBUG:gensim.models.keyedvectors:Skipping line #6 with OOV words: computer	keyboard	7.62
DEBUG:gensim.models.keyedvectors:Skipping line #10 with OOV words: telephone	communication	7.50
DEBUG:gensim.models.keyedvectors:Skipping line #13 with OOV words: drug	abuse	6.85
DEBUG:gensim.models.keyedvectors:Skipping line #14 with OOV words: bread	butter	6.19
DEBUG:gensim.models.keyedvectors:Skipping line #15 with OOV words: cucumber	potato	5.92
DEBUG:gensim.models.keyedv

DEBUG:gensim.models.keyedvectors:Skipping line #133 with OOV words: planet	galaxy	8.11
DEBUG:gensim.models.keyedvectors:Skipping line #135 with OOV words: planet	astronomer	7.94
DEBUG:gensim.models.keyedvectors:Skipping line #136 with OOV words: precedent	example	5.85
DEBUG:gensim.models.keyedvectors:Skipping line #137 with OOV words: precedent	information	3.85
DEBUG:gensim.models.keyedvectors:Skipping line #138 with OOV words: precedent	cognition	2.81
DEBUG:gensim.models.keyedvectors:Skipping line #139 with OOV words: precedent	law	6.65
DEBUG:gensim.models.keyedvectors:Skipping line #140 with OOV words: precedent	collection	2.50
DEBUG:gensim.models.keyedvectors:Skipping line #141 with OOV words: precedent	group	1.77
DEBUG:gensim.models.keyedvectors:Skipping line #142 with OOV words: precedent	antecedent	6.04
DEBUG:gensim.models.keyedvectors:Skipping line #143 with OOV words: cup	coffee	6.58
DEBUG:gensim.models.keyedvectors:Skipping line #144 with OOV words: cup	tableware	6.85
DEBUG:ge

DEBUG:gensim.models.keyedvectors:Skipping line #310 with OOV words: murder	manslaughter	8.53
DEBUG:gensim.models.keyedvectors:Skipping line #311 with OOV words: soap	opera	7.94
DEBUG:gensim.models.keyedvectors:Skipping line #313 with OOV words: life	lesson	5.94
DEBUG:gensim.models.keyedvectors:Skipping line #317 with OOV words: lover	quarrel	6.19
DEBUG:gensim.models.keyedvectors:Skipping line #318 with OOV words: viewer	serial	2.97
DEBUG:gensim.models.keyedvectors:Skipping line #321 with OOV words: morality	importance	3.31
DEBUG:gensim.models.keyedvectors:Skipping line #322 with OOV words: morality	marriage	3.69
DEBUG:gensim.models.keyedvectors:Skipping line #324 with OOV words: gender	equality	6.41
DEBUG:gensim.models.keyedvectors:Skipping line #328 with OOV words: sugar	approach	0.88
DEBUG:gensim.models.keyedvectors:Skipping line #332 with OOV words: size	prominence	5.31
DEBUG:gensim.models.keyedvectors:Skipping line #333 with OOV words: country	citizen	7.31
DEBUG:gensim.models.keyed

DEBUG:gensim.models.keyedvectors:Skipping line #93 with OOV words: unhappy	mad	5.95
DEBUG:gensim.models.keyedvectors:Skipping line #94 with OOV words: sad	terrible	5.4
DEBUG:gensim.models.keyedvectors:Skipping line #95 with OOV words: sick	crazy	3.57
DEBUG:gensim.models.keyedvectors:Skipping line #96 with OOV words: violent	angry	6.98
DEBUG:gensim.models.keyedvectors:Skipping line #97 with OOV words: laden	heavy	5.9
DEBUG:gensim.models.keyedvectors:Skipping line #98 with OOV words: dirty	cheap	1.6
DEBUG:gensim.models.keyedvectors:Skipping line #99 with OOV words: elastic	flexible	7.78
DEBUG:gensim.models.keyedvectors:Skipping line #100 with OOV words: hard	dense	5.9
DEBUG:gensim.models.keyedvectors:Skipping line #102 with OOV words: bold	proud	3.97
DEBUG:gensim.models.keyedvectors:Skipping line #103 with OOV words: sly	strange	1.97
DEBUG:gensim.models.keyedvectors:Skipping line #104 with OOV words: strange	sly	2.07
DEBUG:gensim.models.keyedvectors:Skipping line #105 with OOV words: dum

DEBUG:gensim.models.keyedvectors:Skipping line #224 with OOV words: parent	adult	5.37
DEBUG:gensim.models.keyedvectors:Skipping line #225 with OOV words: bar	jail	1.9
DEBUG:gensim.models.keyedvectors:Skipping line #227 with OOV words: dictionary	definition	6.25
DEBUG:gensim.models.keyedvectors:Skipping line #228 with OOV words: door	cellar	1.97
DEBUG:gensim.models.keyedvectors:Skipping line #229 with OOV words: army	legion	5.95
DEBUG:gensim.models.keyedvectors:Skipping line #230 with OOV words: metal	aluminum	7.25
DEBUG:gensim.models.keyedvectors:Skipping line #231 with OOV words: chair	bench	6.67
DEBUG:gensim.models.keyedvectors:Skipping line #232 with OOV words: cloud	fog	6
DEBUG:gensim.models.keyedvectors:Skipping line #235 with OOV words: bed	blanket	3.02
DEBUG:gensim.models.keyedvectors:Skipping line #236 with OOV words: attorney	lawyer	9.35
DEBUG:gensim.models.keyedvectors:Skipping line #239 with OOV words: clothes	fabric	5.87
DEBUG:gensim.models.keyedvectors:Skipping line #240 w

...

DEBUG:gensim.models.keyedvectors:Skipping line #19133 with OOV words: scream screams vanish vanishes
DEBUG:gensim.models.keyedvectors:Skipping line #19134 with OOV words: scream screams walk walks
DEBUG:gensim.models.keyedvectors:Skipping line #19135 with OOV words: scream screams work works
DEBUG:gensim.models.keyedvectors:Skipping line #19136 with OOV words: scream screams write writes
DEBUG:gensim.models.keyedvectors:Skipping line #19137 with OOV words: scream screams decrease decreases
DEBUG:gensim.models.keyedvectors:Skipping line #19138 with OOV words: scream screams describe describes
DEBUG:gensim.models.keyedvectors:Skipping line #19139 with OOV words: scream screams eat eats
DEBUG:gensim.models.keyedvectors:Skipping line #19140 with OOV words: scream screams enhance enhances
DEBUG:gensim.models.keyedvectors:Skipping line #19141 with OOV words: scream screams estimate estimates
DEBUG:gensim.models.keyedvectors:Skipping line #19142 with OOV words: scream screams find finds
DEBUG

DEBUG:gensim.models.keyedvectors:Skipping line #19218 with OOV words: shuffle shuffles walk walks
DEBUG:gensim.models.keyedvectors:Skipping line #19219 with OOV words: shuffle shuffles work works
DEBUG:gensim.models.keyedvectors:Skipping line #19220 with OOV words: shuffle shuffles write writes
DEBUG:gensim.models.keyedvectors:Skipping line #19221 with OOV words: shuffle shuffles decrease decreases
DEBUG:gensim.models.keyedvectors:Skipping line #19222 with OOV words: shuffle shuffles describe describes
DEBUG:gensim.models.keyedvectors:Skipping line #19223 with OOV words: shuffle shuffles eat eats
DEBUG:gensim.models.keyedvectors:Skipping line #19224 with OOV words: shuffle shuffles enhance enhances
DEBUG:gensim.models.keyedvectors:Skipping line #19225 with OOV words: shuffle shuffles estimate estimates
DEBUG:gensim.models.keyedvectors:Skipping line #19226 with OOV words: shuffle shuffles find finds
DEBUG:gensim.models.keyedvectors:Skipping line #19227 with OOV words: shuffle shuffles g

DEBUG:gensim.models.keyedvectors:Skipping line #19304 with OOV words: slow slows write writes
DEBUG:gensim.models.keyedvectors:Skipping line #19305 with OOV words: slow slows decrease decreases
DEBUG:gensim.models.keyedvectors:Skipping line #19306 with OOV words: slow slows describe describes
DEBUG:gensim.models.keyedvectors:Skipping line #19307 with OOV words: slow slows eat eats
DEBUG:gensim.models.keyedvectors:Skipping line #19308 with OOV words: slow slows enhance enhances
DEBUG:gensim.models.keyedvectors:Skipping line #19309 with OOV words: slow slows estimate estimates
DEBUG:gensim.models.keyedvectors:Skipping line #19310 with OOV words: slow slows find finds
DEBUG:gensim.models.keyedvectors:Skipping line #19311 with OOV words: slow slows generate generates
DEBUG:gensim.models.keyedvectors:Skipping line #19312 with OOV words: slow slows go goes
DEBUG:gensim.models.keyedvectors:Skipping line #19313 with OOV words: slow slows implement implements
DEBUG:gensim.models.keyedvectors:Sk

DEBUG:gensim.models.keyedvectors:Skipping line #19390 with OOV words: talk talks describe describes
DEBUG:gensim.models.keyedvectors:Skipping line #19391 with OOV words: talk talks eat eats
DEBUG:gensim.models.keyedvectors:Skipping line #19392 with OOV words: talk talks enhance enhances
DEBUG:gensim.models.keyedvectors:Skipping line #19393 with OOV words: talk talks estimate estimates
DEBUG:gensim.models.keyedvectors:Skipping line #19394 with OOV words: talk talks find finds
DEBUG:gensim.models.keyedvectors:Skipping line #19395 with OOV words: talk talks generate generates
DEBUG:gensim.models.keyedvectors:Skipping line #19396 with OOV words: talk talks go goes
DEBUG:gensim.models.keyedvectors:Skipping line #19397 with OOV words: talk talks implement implements
DEBUG:gensim.models.keyedvectors:Skipping line #19398 with OOV words: talk talks increase increases
DEBUG:gensim.models.keyedvectors:Skipping line #19399 with OOV words: talk talks listen listens
DEBUG:gensim.models.keyedvectors:

DEBUG:gensim.models.keyedvectors:Skipping line #19475 with OOV words: walk walks eat eats
DEBUG:gensim.models.keyedvectors:Skipping line #19476 with OOV words: walk walks enhance enhances
DEBUG:gensim.models.keyedvectors:Skipping line #19477 with OOV words: walk walks estimate estimates
DEBUG:gensim.models.keyedvectors:Skipping line #19478 with OOV words: walk walks find finds
DEBUG:gensim.models.keyedvectors:Skipping line #19479 with OOV words: walk walks generate generates
DEBUG:gensim.models.keyedvectors:Skipping line #19480 with OOV words: walk walks go goes
DEBUG:gensim.models.keyedvectors:Skipping line #19481 with OOV words: walk walks implement implements
DEBUG:gensim.models.keyedvectors:Skipping line #19482 with OOV words: walk walks increase increases
DEBUG:gensim.models.keyedvectors:Skipping line #19483 with OOV words: walk walks listen listens
DEBUG:gensim.models.keyedvectors:Skipping line #19484 with OOV words: walk walks play plays
DEBUG:gensim.models.keyedvectors:Skipping

In [54]:
google_metrics['benchmarks']

{'test': ['wordsim353', 'wordsim353', 'simlex999', 'simlex999', 'google'],
 'metric': ['pearson_cor',
  'spearman_cor',
  'pearson_cor',
  'spearman_cor',
  'accuracy'],
 'value': [0.2048243359275537,
  0.19803221837418936,
  0.1060918414745899,
  0.10769788024155255,
  0.004214328717639976]}

## Hash Embedding

In [6]:
# conf = global_conf['experiments']['google_default']

In [7]:
from testw2v import google_example
from testw2v.hash_embedding import hash_embedding, hash_embedding_verbose

In [8]:
# dataset, vector_layer = google_example.file_to_dataset(file, conf)

100%|██████████| 18013/18013 [00:04<00:00, 4002.89it/s]


In [55]:
conf

{'batch_size': 64,
 'buffer_size': 10000,
 'embedding_dim': 128,
 'epochs': 20,
 'num_ns': 10,
 'seed': 42,
 'sequence_length': 100,
 'window_size': 4,
 'vocab_size': 4096}

In [13]:
import datetime 

logs = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '25,35')


In [31]:
import importlib
importlib.reload(hash_embedding)

<module 'testw2v.hash_embedding.hash_embedding' from '../testw2v/hash_embedding/hash_embedding.py'>

In [56]:
class Word2Vec(tf.keras.Model):
  def __init__(self, embedding_width, num_words=1024, num_hash_buckets=2**20, num_hash_func=3):
    super(Word2Vec, self).__init__()

    self.target_embedding = hash_embedding.HashEmbedding(num_hash_func=num_hash_func, 
            num_words=num_words, 
            num_hash_buckets=num_hash_buckets, # ~1MM 
            embedding_width=embedding_width, 
            random_seed=1139, 
            name="w2v_target_embedding") 
 
    self.context_embedding = hash_embedding.HashEmbedding(num_hash_func=num_hash_func, 
            num_words=num_words, 
            num_hash_buckets=num_hash_buckets, # ~1MM 
            embedding_width=embedding_width, 
            random_seed=1139, 
            name="w2v_context_embedding") 
 
    self.dots = tf.keras.layers.Dot(axes=(3,2),dtype=tf.float32) 
    self.flatten = tf.keras.layers.Flatten() 
 
 
  def call(self, pair): 
    target, context = pair 
    we = self.target_embedding(target) 
    ce = self.context_embedding(context) 
    dots = self.dots([ce, we]) 
    return self.flatten(dots)


In [57]:
def build_model():
    embedding_dim = conf['embedding_dim'] 
    word2vec = Word2Vec(num_words=conf['vocab_size'], embedding_width=embedding_dim)#, num_hash_buckets=1023)
    word2vec.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return word2vec

model = build_model()

In [ ]:
model.fit(dataset, epochs=20)#10, callbacks=[tboard_callback])

Epoch 1/20
326/326 [==============================] - 854s 3s/step - loss: 1.5983 - accuracy: 0.3173
Epoch 2/20
326/326 [==============================] - 849s 3s/step - loss: 1.4120 - accuracy: 0.4208
Epoch 3/20
326/326 [==============================] - 837s 3s/step - loss: 1.3484 - accuracy: 0.4461
Epoch 4/20
326/326 [==============================] - 829s 3s/step - loss: 1.2929 - accuracy: 0.4787
Epoch 5/20
326/326 [==============================] - 828s 3s/step - loss: 1.2359 - accuracy: 0.5091
Epoch 6/20
326/326 [==============================] - 820s 3s/step - loss: 1.1758 - accuracy: 0.5389
Epoch 7/20
326/326 [==============================] - 819s 3s/step - loss: 1.1120 - accuracy: 0.5686
Epoch 8/20
326/326 [==============================] - 816s 3s/step - loss: 1.0445 - accuracy: 0.5985
Epoch 9/20
326/326 [==============================] - 812s 2s/step - loss: 0.9733 - accuracy: 0.6300
Epoch 10/20
326/326 [==============================] - 819s 3s/step - loss: 0.8984 - accura

In [59]:
vocab = vector_layer.get_vocabulary()

In [60]:
emb = model.layers[0](tf.constant([range(len(vocab))]))

In [61]:
weights = tf.squeeze(emb).numpy()

In [62]:
# vocab = vector_layer.get_vocabulary()
# weights = model.get_layer('w2v_embedding').get_weights()[0]

gensim_obj = gensim_utils.w2v_to_gensim(vocab, weights)
google_metrics = gensim_utils.metrics(gensim_obj)

INFO:gensim.models.keyedvectors:precomputing L2-norms of word weight vectors
DEBUG:smart_open.smart_open_lib:{'transport_params': None, 'ignore_ext': False, 'opener': None, 'closefd': True, 'newline': None, 'errors': None, 'encoding': None, 'buffering': -1, 'mode': 'rb', 'uri': '/opt/conda/envs/emb3/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv'}
DEBUG:gensim.models.keyedvectors:Skipping line #3 with OOV words: tiger	cat	7.35
DEBUG:gensim.models.keyedvectors:Skipping line #4 with OOV words: tiger	tiger	10.00
DEBUG:gensim.models.keyedvectors:Skipping line #6 with OOV words: computer	keyboard	7.62
DEBUG:gensim.models.keyedvectors:Skipping line #10 with OOV words: telephone	communication	7.50
DEBUG:gensim.models.keyedvectors:Skipping line #13 with OOV words: drug	abuse	6.85
DEBUG:gensim.models.keyedvectors:Skipping line #14 with OOV words: bread	butter	6.19
DEBUG:gensim.models.keyedvectors:Skipping line #15 with OOV words: cucumber	potato	5.92
DEBUG:gensim.models.keyedv

DEBUG:gensim.models.keyedvectors:Skipping line #133 with OOV words: planet	galaxy	8.11
DEBUG:gensim.models.keyedvectors:Skipping line #135 with OOV words: planet	astronomer	7.94
DEBUG:gensim.models.keyedvectors:Skipping line #136 with OOV words: precedent	example	5.85
DEBUG:gensim.models.keyedvectors:Skipping line #137 with OOV words: precedent	information	3.85
DEBUG:gensim.models.keyedvectors:Skipping line #138 with OOV words: precedent	cognition	2.81
DEBUG:gensim.models.keyedvectors:Skipping line #139 with OOV words: precedent	law	6.65
DEBUG:gensim.models.keyedvectors:Skipping line #140 with OOV words: precedent	collection	2.50
DEBUG:gensim.models.keyedvectors:Skipping line #141 with OOV words: precedent	group	1.77
DEBUG:gensim.models.keyedvectors:Skipping line #142 with OOV words: precedent	antecedent	6.04
DEBUG:gensim.models.keyedvectors:Skipping line #143 with OOV words: cup	coffee	6.58
DEBUG:gensim.models.keyedvectors:Skipping line #144 with OOV words: cup	tableware	6.85
DEBUG:ge

DEBUG:gensim.models.keyedvectors:Skipping line #310 with OOV words: murder	manslaughter	8.53
DEBUG:gensim.models.keyedvectors:Skipping line #311 with OOV words: soap	opera	7.94
DEBUG:gensim.models.keyedvectors:Skipping line #313 with OOV words: life	lesson	5.94
DEBUG:gensim.models.keyedvectors:Skipping line #317 with OOV words: lover	quarrel	6.19
DEBUG:gensim.models.keyedvectors:Skipping line #318 with OOV words: viewer	serial	2.97
DEBUG:gensim.models.keyedvectors:Skipping line #321 with OOV words: morality	importance	3.31
DEBUG:gensim.models.keyedvectors:Skipping line #322 with OOV words: morality	marriage	3.69
DEBUG:gensim.models.keyedvectors:Skipping line #324 with OOV words: gender	equality	6.41
DEBUG:gensim.models.keyedvectors:Skipping line #328 with OOV words: sugar	approach	0.88
DEBUG:gensim.models.keyedvectors:Skipping line #332 with OOV words: size	prominence	5.31
DEBUG:gensim.models.keyedvectors:Skipping line #333 with OOV words: country	citizen	7.31
DEBUG:gensim.models.keyed

DEBUG:gensim.models.keyedvectors:Skipping line #93 with OOV words: unhappy	mad	5.95
DEBUG:gensim.models.keyedvectors:Skipping line #94 with OOV words: sad	terrible	5.4
DEBUG:gensim.models.keyedvectors:Skipping line #95 with OOV words: sick	crazy	3.57
DEBUG:gensim.models.keyedvectors:Skipping line #96 with OOV words: violent	angry	6.98
DEBUG:gensim.models.keyedvectors:Skipping line #97 with OOV words: laden	heavy	5.9
DEBUG:gensim.models.keyedvectors:Skipping line #98 with OOV words: dirty	cheap	1.6
DEBUG:gensim.models.keyedvectors:Skipping line #99 with OOV words: elastic	flexible	7.78
DEBUG:gensim.models.keyedvectors:Skipping line #100 with OOV words: hard	dense	5.9
DEBUG:gensim.models.keyedvectors:Skipping line #102 with OOV words: bold	proud	3.97
DEBUG:gensim.models.keyedvectors:Skipping line #103 with OOV words: sly	strange	1.97
DEBUG:gensim.models.keyedvectors:Skipping line #104 with OOV words: strange	sly	2.07
DEBUG:gensim.models.keyedvectors:Skipping line #105 with OOV words: dum

DEBUG:gensim.models.keyedvectors:Skipping line #224 with OOV words: parent	adult	5.37
DEBUG:gensim.models.keyedvectors:Skipping line #225 with OOV words: bar	jail	1.9
DEBUG:gensim.models.keyedvectors:Skipping line #227 with OOV words: dictionary	definition	6.25
DEBUG:gensim.models.keyedvectors:Skipping line #228 with OOV words: door	cellar	1.97
DEBUG:gensim.models.keyedvectors:Skipping line #229 with OOV words: army	legion	5.95
DEBUG:gensim.models.keyedvectors:Skipping line #230 with OOV words: metal	aluminum	7.25
DEBUG:gensim.models.keyedvectors:Skipping line #231 with OOV words: chair	bench	6.67
DEBUG:gensim.models.keyedvectors:Skipping line #232 with OOV words: cloud	fog	6
DEBUG:gensim.models.keyedvectors:Skipping line #235 with OOV words: bed	blanket	3.02
DEBUG:gensim.models.keyedvectors:Skipping line #236 with OOV words: attorney	lawyer	9.35
DEBUG:gensim.models.keyedvectors:Skipping line #239 with OOV words: clothes	fabric	5.87
DEBUG:gensim.models.keyedvectors:Skipping line #240 w

...

DEBUG:gensim.models.keyedvectors:see sees provide provides: expected PROVIDES, predicted VESSEL
DEBUG:gensim.models.keyedvectors:see sees say says: expected SAYS, predicted WRITES
DEBUG:gensim.models.keyedvectors:Skipping line #19208 with OOV words: see sees scream screams
DEBUG:gensim.models.keyedvectors:Skipping line #19209 with OOV words: see sees search searches
DEBUG:gensim.models.keyedvectors:Skipping line #19210 with OOV words: shuffle shuffles sing sings
DEBUG:gensim.models.keyedvectors:Skipping line #19211 with OOV words: shuffle shuffles sit sits
DEBUG:gensim.models.keyedvectors:Skipping line #19212 with OOV words: shuffle shuffles slow slows
DEBUG:gensim.models.keyedvectors:Skipping line #19213 with OOV words: shuffle shuffles speak speaks
DEBUG:gensim.models.keyedvectors:Skipping line #19214 with OOV words: shuffle shuffles swim swims
DEBUG:gensim.models.keyedvectors:Skipping line #19215 with OOV words: shuffle shuffles talk talks
DEBUG:gensim.models.keyedvectors:Skipping l

DEBUG:gensim.models.keyedvectors:Skipping line #19291 with OOV words: sit sits say says
DEBUG:gensim.models.keyedvectors:Skipping line #19292 with OOV words: sit sits scream screams
DEBUG:gensim.models.keyedvectors:Skipping line #19293 with OOV words: sit sits search searches
DEBUG:gensim.models.keyedvectors:Skipping line #19294 with OOV words: sit sits see sees
DEBUG:gensim.models.keyedvectors:Skipping line #19295 with OOV words: sit sits shuffle shuffles
DEBUG:gensim.models.keyedvectors:Skipping line #19296 with OOV words: sit sits sing sings
DEBUG:gensim.models.keyedvectors:Skipping line #19297 with OOV words: slow slows speak speaks
DEBUG:gensim.models.keyedvectors:Skipping line #19298 with OOV words: slow slows swim swims
DEBUG:gensim.models.keyedvectors:Skipping line #19299 with OOV words: slow slows talk talks
DEBUG:gensim.models.keyedvectors:Skipping line #19300 with OOV words: slow slows think thinks
DEBUG:gensim.models.keyedvectors:Skipping line #19301 with OOV words: slow sl

DEBUG:gensim.models.keyedvectors:Skipping line #19377 with OOV words: swim swims search searches
DEBUG:gensim.models.keyedvectors:Skipping line #19378 with OOV words: swim swims see sees
DEBUG:gensim.models.keyedvectors:Skipping line #19379 with OOV words: swim swims shuffle shuffles
DEBUG:gensim.models.keyedvectors:Skipping line #19380 with OOV words: swim swims sing sings
DEBUG:gensim.models.keyedvectors:Skipping line #19381 with OOV words: swim swims sit sits
DEBUG:gensim.models.keyedvectors:Skipping line #19382 with OOV words: swim swims slow slows
DEBUG:gensim.models.keyedvectors:Skipping line #19383 with OOV words: swim swims speak speaks
DEBUG:gensim.models.keyedvectors:Skipping line #19384 with OOV words: talk talks think thinks
DEBUG:gensim.models.keyedvectors:Skipping line #19385 with OOV words: talk talks vanish vanishes
DEBUG:gensim.models.keyedvectors:Skipping line #19386 with OOV words: talk talks walk walks
DEBUG:gensim.models.keyedvectors:Skipping line #19387 with OOV w

DEBUG:gensim.models.keyedvectors:Skipping line #19462 with OOV words: vanish vanishes see sees
DEBUG:gensim.models.keyedvectors:Skipping line #19463 with OOV words: vanish vanishes shuffle shuffles
DEBUG:gensim.models.keyedvectors:Skipping line #19464 with OOV words: vanish vanishes sing sings
DEBUG:gensim.models.keyedvectors:Skipping line #19465 with OOV words: vanish vanishes sit sits
DEBUG:gensim.models.keyedvectors:Skipping line #19466 with OOV words: vanish vanishes slow slows
DEBUG:gensim.models.keyedvectors:Skipping line #19467 with OOV words: vanish vanishes speak speaks
DEBUG:gensim.models.keyedvectors:Skipping line #19468 with OOV words: vanish vanishes swim swims
DEBUG:gensim.models.keyedvectors:Skipping line #19469 with OOV words: vanish vanishes talk talks
DEBUG:gensim.models.keyedvectors:Skipping line #19470 with OOV words: vanish vanishes think thinks
DEBUG:gensim.models.keyedvectors:Skipping line #19471 with OOV words: walk walks work works
DEBUG:gensim.models.keyedvect

DEBUG:gensim.models.keyedvectors:Skipping line #19548 with OOV words: write writes sing sings
DEBUG:gensim.models.keyedvectors:Skipping line #19549 with OOV words: write writes sit sits
DEBUG:gensim.models.keyedvectors:Skipping line #19550 with OOV words: write writes slow slows
DEBUG:gensim.models.keyedvectors:Skipping line #19551 with OOV words: write writes speak speaks
DEBUG:gensim.models.keyedvectors:Skipping line #19552 with OOV words: write writes swim swims
DEBUG:gensim.models.keyedvectors:Skipping line #19553 with OOV words: write writes talk talks
DEBUG:gensim.models.keyedvectors:Skipping line #19554 with OOV words: write writes think thinks
DEBUG:gensim.models.keyedvectors:Skipping line #19555 with OOV words: write writes vanish vanishes
DEBUG:gensim.models.keyedvectors:Skipping line #19556 with OOV words: write writes walk walks
DEBUG:gensim.models.keyedvectors:write writes work works: expected WORKS, predicted COLONEL
INFO:gensim.models.keyedvectors:gram9-plural-verbs: 0.0

In [63]:
google_metrics['benchmarks']

{'test': ['wordsim353', 'wordsim353', 'simlex999', 'simlex999', 'google'],
 'metric': ['pearson_cor',
  'spearman_cor',
  'pearson_cor',
  'spearman_cor',
  'accuracy'],
 'value': [0.20440159169903446,
  0.22696310818083573,
  0.07731101727505464,
  0.06610757185106098,
  0.0012040939193257074]}

In [64]:
google_metrics

{'french_similarity': {'word': ['stanley',
   'portuguese',
   'santa',
   'sides',
   'representative',
   'hundred',
   'bid',
   'design',
   'colony',
   'leads'],
  'cosine_dist': [0.4177094101905823,
   0.39399996399879456,
   0.36912304162979126,
   0.36698660254478455,
   0.35873448848724365,
   0.3585202097892761,
   0.3582533001899719,
   0.34804871678352356,
   0.3477400243282318,
   0.34544846415519714]},
 'december_similarity': {'word': ['february',
   'april',
   'march',
   'lap',
   'july',
   'january',
   'september',
   'door',
   'beginning',
   'november'],
  'cosine_dist': [0.5689401626586914,
   0.5432120561599731,
   0.5217983722686768,
   0.5060099363327026,
   0.5013777017593384,
   0.4951739013195038,
   0.49272727966308594,
   0.48103412985801697,
   0.46789631247520447,
   0.46769076585769653]},
 'street_similarity': {'word': ['memorial',
   'victoria',
   '46',
   'historian',
   'athletic',
   'camera',
   'hotel',
   'dr',
   'temple',
   'market'],
  'c